In [57]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nqDataLoader as nq #data loading library
from keras.preprocessing import sequence

In [58]:
## importing the early stage dataset 
early_stage = pd.read_csv('GT_DataPD_MIT-CS2PD.csv')
# X = dataset.iloc[:, :-1].values
# y = dataset.iloc[:, -1].values
early_stage["gt"] = early_stage["gt"].astype(int)
early_stage.head()

,pID,gt,updrs108,afTap,sTap,nqScore,typingSpeed,file_1
0,1000,1,27,79.0,184.5,0.107179,56.866667,1424946827.1000_001_014.csv
1,1001,1,16,96.5,189.0,0.056286,118.000000,1427279751.1001_001_014.csv
2,1002,0,5,140.0,158.0,0.039519,119.037037,1426676689.1002_001_014.csv
3,1004,1,22,83.5,191.5,0.034853,74.266667,1429866367.1004_001_014.csv
4,1005,1,17,68.0,150.0,0.048307,74.969697,1430134526.1005_001_014.csv


In [59]:
hit_time_series = []
for index, row in early_stage.iterrows():
    fileloc = row.file_1
    keyPressed, htArr, pressArr, releaseArr =  nq.getDataFiltHelper( "data_MIT-CS2PD/" + early_stage.loc[index]['file_1'])
    htArr =np.array(htArr)
    hit_time_series.append(htArr)

X1 = hit_time_series 

In [60]:
## importing the de-novo dataset 
de_novo = pd.read_csv('GT_DataPD_MIT-CS1PD.csv')
# X = dataset.iloc[:, :-1].values
# y = dataset.iloc[:, -1].values
print(len(de_novo))
de_novo["gt"] = de_novo["gt"].astype(int)
de_novo.head()

31


,pID,gt,updrs108,afTap,sTap,nqScore,typingSpeed,file_1,file_2
0,11,1,14.25,NaN,162.25,0.117543,189.372549,1402930351.011_001_014.csv,1403706430.011_003_014.csv
1,60,0,2.00,NaN,162.25,0.070350,60.533333,1402932300.060_001_014.csv,1403708258.060_003_014.csv
2,67,1,25.25,NaN,133.75,0.223411,54.333333,1401117235.067_001_014.csv,1401978395.067_003_014.csv
3,68,0,6.00,NaN,159.00,0.074973,71.800000,1401114972.068_001_014.csv,1401980765.068_003_014.csv
4,70,1,26.25,NaN,113.50,0.175751,39.614035,1404311419.070_001_014.csv,1404743687.070_003_014.csv


In [61]:
##using both files 
hit_time_series = []
for index, row in de_novo.iterrows():
    fileloc1 = row.file_1
    keyPressed, htArr1, pressArr, releaseArr =  nq.getDataFiltHelper( 'data_MIT-CS1PD/' + de_novo.loc[index]['file_1'])
    htArr1 = np.array(htArr1)
    keyPressed, htArr2, pressArr, releaseArr =  nq.getDataFiltHelper( 'data_MIT-CS1PD/' + de_novo.loc[index]['file_2'])
    htArr2 = np.array(htArr2)
    htArr =np.concatenate((htArr1,htArr2),axis =0)
    htArr=np.array(htArr)
    hit_time_series.append(htArr)
X2 = hit_time_series 

In [62]:
X = np.concatenate((X1,X2),axis=0)
y=  np.concatenate((early_stage['gt'],de_novo["gt"]),axis=0)
X_filtered =[]
y_filtered =[]
for i,e in enumerate(X):
    if len(e)>1000:
        X_filtered.append(e)
        y_filtered.append(y[i])
min = len(X_filtered[0])
for i,e in enumerate(X_filtered):
    if len(e)<min:
        min = len(e)
        print(i)
X_filtered = np.array(X_filtered)
y_filtered = np.array(y_filtered)
print(min)
print(len(X_filtered),len(y_filtered))

2
6
12
58
1006
73 73


<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
C:\Users\micoa\AppData\Local\Temp\ipykernel_7104\1946120355.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_filtered = np.array(X_filtered)


In [63]:
X_filtered =sequence.pad_sequences(X_filtered,dtype='float32',truncating='post',maxlen=min) 



In [64]:
from pandas import DataFrame
from pandas import concat
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [65]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_filtered,y_filtered,test_size=0.2)

In [66]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
X_train = sc.fit_transform(X_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
print(X_train)
X_test = sc.transform(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

[[[0.09379071]
  [0.17947493]
  [0.04386264]
  ...
  [0.3835553 ]
  [0.4301293 ]
  [0.18871158]]

 [[0.4628746 ]
  [0.125537  ]
  [0.19789188]
  ...
  [0.04134129]
  [0.09127626]
  [0.17787915]]

 [[0.2952671 ]
  [0.4673031 ]
  [0.47330844]
  ...
  [0.48139638]
  [0.29765755]
  [0.6174459 ]]

 ...

 [[0.21189752]
  [0.33365154]
  [0.11186673]
  ...
  [0.20257233]
  [0.17730212]
  [0.23489171]]

 [[0.3595311 ]
  [0.69021475]
  [0.54029244]
  ...
  [0.96508956]
  [1.0000001 ]
  [0.90250856]]

 [[0.04559273]
  [0.11455847]
  [0.29037744]
  ...
  [0.05925585]
  [0.14983845]
  [0.05131128]]]


In [67]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
clf = Sequential()
clf.add(LSTM(units = 100,return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
clf.add(Dropout(0.2))
clf.add(LSTM(units = 100,return_sequences = True))
clf.add(Dropout(0.2))
clf.add(LSTM(units = 100))
clf.add(Dropout(0.2))
clf.add(Dense(units = 1,activation='sigmoid'))
clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(clf.summary())
clf.fit(X_train, y_train, epochs=100, batch_size=5)
scores = clf.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 1006, 100)         40800     
                                                                 
 dropout_6 (Dropout)         (None, 1006, 100)         0         
                                                                 
 lstm_7 (LSTM)               (None, 1006, 100)         80400     
                                                                 
 dropout_7 (Dropout)         (None, 1006, 100)         0         
                                                                 
 lstm_8 (LSTM)               (None, 100)               80400     
                                                                 
 dropout_8 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                

12/12 [==============================] - 21s 2s/step - loss: 0.6779 - accuracy: 0.5517
Epoch 70/100
12/12 [==============================] - 21s 2s/step - loss: 0.6891 - accuracy: 0.5000
Epoch 71/100
12/12 [==============================] - 21s 2s/step - loss: 0.7034 - accuracy: 0.4138
Epoch 72/100
12/12 [==============================] - 21s 2s/step - loss: 0.6751 - accuracy: 0.4138
Epoch 73/100
12/12 [==============================] - 21s 2s/step - loss: 0.6794 - accuracy: 0.5345
Epoch 74/100
12/12 [==============================] - 21s 2s/step - loss: 0.6752 - accuracy: 0.4655
Epoch 75/100
12/12 [==============================] - 21s 2s/step - loss: 0.6537 - accuracy: 0.6379
Epoch 76/100
12/12 [==============================] - 21s 2s/step - loss: 0.6843 - accuracy: 0.4138
Epoch 77/100
12/12 [==============================] - 21s 2s/step - loss: 0.6608 - accuracy: 0.5690
Epoch 78/100
12/12 [==============================] - 21s 2s/step - loss: 0.6617 - accuracy: 0.4828
Epoch 79/100
